In [17]:
import torch
import torchvision
import torchvision.transforms as transforms

# 데이터셋 로딩 및 전처리
transform = transforms.Compose(
    [transforms.ToTensor(),  # 이미지를 텐서로 변환. 컴터가 알아먹어야하니까
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])  # 이미지를 정규화. 데이터 분포의 표준화를 위해! 
    #다양한 데이터 특징에 덜 민감하게 만들 수 있어서 일반화 성능 향상. 오버피팅 방지

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)  # 학습 데이터셋 로딩
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4,
                                          shuffle=True, num_workers=2)  # 데이터 로더 생성

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)  # 테스트 데이터셋 로딩
testloader = torch.utils.data.DataLoader(testset, batch_size=4,
                                         shuffle=False, num_workers=2)  # 데이터 로더 생성



Files already downloaded and verified
Files already downloaded and verified


In [18]:
import torch.nn as nn
import torch.nn.functional as F

# 신경망 정의
class Net(nn.Module):
    #__init__: 이 함수는 신경망의 구조를 정의
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)  # 3채널(RGB) 입력을  5x5 필터를 사용하여 6채널(6개의 필터)로 컨볼루션하는 레이어
        #6개의 서로 다른 5x5 필터를 각각 입력 이미지에 동시에 적용.
        # 이 각각의 필터는 이미지의 다른 특징을 학습하고 감지. 
        # 예를 들어, 하나의 필터는 가장자리를 감지하도록 학습되고, 
        # 다른 필터는 특정 색상 패턴을 감지하도록 학습 수 있음.

        
        self.pool = nn.MaxPool2d(2, 2)  # 2x2 맥스 풀링. 맥스풀링은 특징 맴 크기 줄이고(다운 샘플링) 2*2중 가장 큰 값 뽑아 특징강조.
        self.conv2 = nn.Conv2d(6, 16, 5)  # 6채널 입력을 16채널로 컨볼루션
        self.fc1 = nn.Linear(16 * 5 * 5, 120)  # 16*5*5 차원의 텐서를 120차원으로 선형 변환
        self.fc2 = nn.Linear(120, 84)  # 120차원을 84차원으로 선형 변환
        self.fc3 = nn.Linear(84, 10)  # 84차원을 10차원으로 선형 변환

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))  # 컨볼루션 -> ReLU -> 맥스 풀링
        x = self.pool(F.relu(self.conv2(x)))  # 컨볼루션 -> ReLU -> 맥스 풀링
        x = x.view(-1, 16 * 5 * 5)  # 3차원 텐서를 1차원으로 평탄화
        x = F.relu(self.fc1(x))  # 선형 변환 -> ReLU
        x = F.relu(self.fc2(x))  # 선형 변환 -> ReLU
        x = self.fc3(x)  # 선형 변환
        return x

net = Net()


In [19]:
import torch.optim as optim

# 손실 함수 및 옵티마이저 설정
criterion = nn.CrossEntropyLoss()  # 크로스 엔트로피 손실 함수
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)  # SGD 옵티마이저 설정

# 모델 훈련
for epoch in range(2):
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data

        optimizer.zero_grad()  # 그래디언트 초기화

        outputs = net(inputs)  # 모델 순전파
        loss = criterion(outputs, labels)  # 손실 계산
        loss.backward()  # 역전파
        optimizer.step()  # 옵티마이저 업데이트

        running_loss += loss.item()
        if i % 2000 == 1999:
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')


[1,  2000] loss: 2.170
[1,  4000] loss: 1.791
[1,  6000] loss: 1.634
[1,  8000] loss: 1.535
[1, 10000] loss: 1.510
[1, 12000] loss: 1.473
[2,  2000] loss: 1.381
[2,  4000] loss: 1.380
[2,  6000] loss: 1.339
[2,  8000] loss: 1.317
[2, 10000] loss: 1.318
[2, 12000] loss: 1.290
Finished Training


In [20]:
# 모델 테스트
correct = 0  # 정확하게 예측한 이미지의 수를 저장하는 변수 초기화
total = 0  # 전체 이미지의 수를 저장하는 변수 초기화
with torch.no_grad():  # 그라디언트 계산 비활성화
    for data in testloader:  # 테스트 데이터셋을 순회
        images, labels = data  # 이미지와 레이블을 가져옴
        outputs = net(images)  # 이미지를 모델에 입력하여 출력을 얻음
        _, predicted = torch.max(outputs.data, 1)  # 가장 높은 확률을 가진 클래스를 선택
        total += labels.size(0)  # 전체 이미지 수 업데이트
        correct += (predicted == labels).sum().item()  # 정확히 예측한 이미지 수 업데이트

# 정확도 출력
print(f'네트워크의 정확도: {100 * correct / total}%')


네트워크의 정확도: 53.51%


: 